# Knowledge Error Detection using K-means and Adaptive Clustering

## 1. Clustering positive entity vectors using k-means
- Attempt to k-means clustering with an entity vector of triples consisting of <entity, type, person>.
- A positive entity means that the entity is a person in the triple of <entity, type, person>.
    - (e.g. entity : 'Biden', 'Trump', ...)
- Find the maximum Euclidean distance between the centroid of each clusters and the elements included in the cluster.
- The maximum Euclidean distance obtained from each cluster means the radius $r$ of the cluster, and entities located between the radius from the centroid are classified as entities of the cluster.

## 2. Apply the adaptive clustering method.
- Add the negative entity vector to the vector space clustered by the positive entity vector.
- A negative entity means that the entity is not a person in the triple of <entity, type, person>.
    - (e.g. entity : 'Titanic'(Film), 'New York'(City), ...)
- Negative vectors are included in the cluster by calculating the distance between each centroid of the cluster.
- Find the optimal $\delta $(0.6 ~ 1.0) that will maximize the $f1-score$ of each cluster.
- The $\delta $ is multiplied by the cluster radius $r$ to create a new cluster range $r'$.
    - $f1-score = \frac{2\times precision\times recall}{precision+recall}$
    - $precision = \frac{TP}{TP+FP}$
    - $recall = \frac{TP}{TP+FN}$
    - $TP(True Positive)$ = Number of positive entities included in the cluster  
    - $FP(False Positive)$ = Number of negative entities included in the cluster
    - $FN(False Negative)$ = The number of positive entities included in the cluster before the cluster range was adjusted but not included after the adjustment.  

## 3. Perform error triple detection.
- Add the positive and negative vectors of the test data to the vector space.
- Apply a new radius $r'$ to each cluster multiplied by the optimal $\delta $.
- Entities included in the cluster are classified as positive entities, and entities not included in any cluster are classified as negative entities.

In [1]:
# custom functions
from AdaClustering.ClusteringAlgorithm import kmeans_alg, grant_to_cluster
from evaluation.metrics import get_initial_TP, get_ada_matrics, get_optimal_matrics
from util.counter import count_element, count_TPFP

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances  #k-means using euclidean_distances
import pickle

## Load GloVe embedding vectors from file
- dataname options :
    - dbpedia / freebase / wisekb

In [2]:
dataname = 'dbpedia'
k_dict = {'freebase' : 46, 'dbpedia' : 15, 'wiseKB' : 27}

opt_k = k_dict[dataname]

f = open(f'./data/GloVeEntityVectors/glove_{dataname}/person_embedding','rb')
vector = pickle.load(f, encoding='latin1')
f.close()

f = open(f'./data/GloVeEntityVectors/glove_{dataname}/person_words','rb')
word = pickle.load(f, encoding='latin1')
f.close()
 
glove_dict = {}
for i in range(len(vector)):
    glove_dict[word[i]] = vector[i]
    
print(len(glove_dict)) 

50664


### load entity label from file

In [3]:
label_dir = f'./data/{dataname}/'

train_pos_loc = label_dir + 'train_positive_20000.txt'
train_neg_loc = label_dir + 'train_negative_5000.txt'
test_pos_loc = label_dir + 'test_positive_5000.txt'
test_neg_loc = label_dir + 'test_negative_5000.txt'

train_pos_embedding = []
train_neg_embedding = []
test_pos_embedding = []
test_neg_embedding = []

if dataname=='freebase':
    sep = '\t'
else:
    sep = ' '
    
with open(train_pos_loc) as f:          
    for i in f:
        train_pos_embedding.append(i.split(sep)[0].strip())
        
print('train_pos : ',len(train_pos_embedding))
print(train_pos_embedding[:5])

with open(train_neg_loc) as f:          
    for i in f:
        train_neg_embedding.append(i.split(sep)[0].strip())
        
print('\ntrain_neg : ',len(train_neg_embedding))
print(train_neg_embedding[:5])

with open(test_pos_loc) as f:       
    for i in f:
        test_pos_embedding.append(i.split(sep)[0].strip())
        
print('\ntest_pos : ',len(test_pos_embedding))
print(test_pos_embedding[:5])

with open(test_neg_loc) as f:
    for i in f:
        test_neg_embedding.append(i.split(sep)[0].strip())
        
print('\ntest_neg : ',len(test_neg_embedding))
print(test_neg_embedding[:5])

train_pos :  20000
['e_2398311', 'e_4856915', 'e_375534', 'e_401503', 'e_1890861']

train_neg :  5000
['e_2022579', 'e_4008488', 'e_1863291', 'e_96735', 'e_2223783']

test_pos :  5000
['e_1089098', 'e_4119140', 'e_2066827', 'e_1183621', 'e_1708557']

test_neg :  5000
['e_1629807', 'e_2163227', 'e_5848230', 'e_2079695', 'e_1725594']


## 1. Clustering positive entity vectors using k-means

In [4]:
vectors, centroids, c_label = kmeans_alg(train_pos_embedding, opt_k, glove_dict)

# dictionary for labeling entity with cluster - key : cluster label / - value : list of vectors
dicts_each_cluster = {}
for c in range(opt_k):
    same_Cluster = []    
    for idx, label in enumerate(c_label):
        if label == c:
            same_Cluster.append(vectors[idx].tolist())    
    dicts_each_cluster[c]= list(same_Cluster)

# dictionary for each cluster's max Euclidean distance 
max_distance_each_cluster = {}

for i in dicts_each_cluster.keys():
    in_vectors = dicts_each_cluster[i] 
    tmp_distance = euclidean_distances(in_vectors, centroids[i].reshape(1,-1))
    
    # max_distance_each_cluster - key : cluster lable / - value : max_distance
    max_distance_each_cluster[i] = max(tmp_distance)[0].astype(np.float32) 
    
# Get vectors 
train_pos_vector = list(map(lambda x: (x,glove_dict[x]), train_pos_embedding)) # [(word, vector), ... (word, vector)]
train_neg_vector = list(map(lambda x: (x,glove_dict[x]), train_neg_embedding)) # [(word, vector), ... (word, vector)]

### Check Statistics before delta optimzing 

In [5]:
true_cluster = grant_to_cluster(train_pos_vector, 1, max_distance_each_cluster, centroids)
false_cluster = grant_to_cluster(train_neg_vector, 1, max_distance_each_cluster, centroids)
true_count, false_count = count_TPFP(centroids, true_cluster, false_cluster)

result_list = []

# 각 클러스터에 포함된 positive 엔티티, negative 엔티티수로 각 클러스터에 대한 precision 지표 산출
for cluster in range(len(centroids)):
    TP = true_count[cluster] # True Positive : 클러스터 포함되어있는 positive 엔티티
    FP = false_count[cluster] # False Positive : 클러스터에 포함되어있는 negative 엔티티
    if (TP+FP) == 0:
        precision = 0.0
    else:
        precision = TP / (TP+FP)
    result_list.append([TP,FP,precision])

# first_result_df
pre_result_df = pd.DataFrame(result_list, columns=['initial_cluster_pos', 
                                                   'initial_cluster_neg', 
                                                   'initial_cluster_precision'])
pre_result_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision
0,1401,248,0.849606
1,1547,92,0.943868
2,1343,265,0.835199
3,1298,211,0.860172
4,1401,280,0.833432


## 2. Apply the adaptive clustering method.

### optimize delta for each clusters

In [6]:
P, R, F1, TP, FN, FP, initial_TP = get_ada_matrics(train_pos_vector, train_neg_vector, 
                                                   max_distance_each_cluster, centroids)


TP_list = TP[np.array(F1).argmax(0),list(range(TP.shape[1]))]

#cluster의 f1 score가 최대가 되는 지점의 delta를 optimal delta로 선정
optimal_deltas = np.array(F1).argmax(0)
optimal_deltas = (optimal_deltas * 0.01)+0.6

# positive entity수가 0인 cluster는 delta를 0으로 변환
optimal_deltas = np.array([d if tp != 0 else 0 for d, tp in zip(optimal_deltas, TP_list)])

print('optimaldeltas each clusters : \n', optimal_deltas)

optimaldeltas each clusters : 
 [0.98 1.   1.   0.96 0.9  1.   0.93 0.96 1.   1.   0.97 0.99 1.   1.
 0.95]


In [7]:
# 구한 optimal delta를 cluster에 적용하여 각 cluster에 대한 precision, recall, TP, FP를 산출
optimalP, optimalR, optimalTP, optimalFP = get_optimal_matrics(train_pos_vector, train_neg_vector, 
                                                               max_distance_each_cluster,centroids, 
                                                               optimal_deltas, initial_TP)

matrics = []
for m1, m2, m3, m4, m5 in zip(optimalP, optimalR,  optimalTP, optimalFP, optimal_deltas):
    matrics.append([m1, m2, m3, m4, m5])
    
opt_delta_df = pd.DataFrame(matrics, columns=['opt_precision', 'opt_recall', 'opt_TP','opt_FP', 'opt_delta'])
tmp_parse_df = opt_delta_df[['opt_delta','opt_TP','opt_FP', 'opt_precision']]
train_result_df = pd.concat([pre_result_df, tmp_parse_df], axis = 1)
train_result_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision,opt_delta,opt_TP,opt_FP,opt_precision
0,1401,248,0.849606,0.98,1399,245,0.850973
1,1547,92,0.943868,1.00,1547,92,0.943868
2,1343,265,0.835199,1.00,1343,265,0.835199
3,1298,211,0.860172,0.96,1295,207,0.862184
4,1401,280,0.833432,0.90,1399,276,0.835224


## 3. Error triple detection using test dataset

In [8]:
# Get test vectors from embedding model 
test_pos_vector = list(map(lambda x: (x,glove_dict[x]), test_pos_embedding))
test_neg_vector = list(map(lambda x: (x,glove_dict[x]), test_neg_embedding))
initial_TP = get_initial_TP(test_pos_vector, max_distance_each_cluster, centroids)

# test 데이터의 embedding 벡터를 cluster에 할당하여 Precision, Recall 산출
optimalP, optimalR, optimalTP, optimalFP = get_optimal_matrics(test_pos_vector, test_neg_vector, 
                                                               max_distance_each_cluster, centroids, 
                                                               optimal_deltas, initial_TP)

In [9]:
matrics = []
for m1, m2, m3, m4, m5 in zip(optimalP, optimalR, optimalTP, optimalFP, optimal_deltas):
    matrics.append([m1, m2, m3, m4, m5])
    
output_df = pd.DataFrame(matrics, columns=['precision', 'recall', 'TP','FP', 'delta'])
tmp_output_df = output_df[['TP','FP', 'precision', 'recall']]
tmp_output_df.columns = ['Test_TP','Test_FP', 'Test_precision', 'Test_recall']

train_test_df = pd.concat([train_result_df, tmp_output_df], axis = 1)
train_test_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision,opt_delta,opt_TP,opt_FP,opt_precision,Test_TP,Test_FP,Test_precision,Test_recall
0,1401,248,0.849606,0.98,1399,245,0.850973,345,262,0.568369,1.000000
1,1547,92,0.943868,1.00,1547,92,0.943868,392,100,0.796748,1.000000
2,1343,265,0.835199,1.00,1343,265,0.835199,348,237,0.594872,1.000000
3,1298,211,0.860172,0.96,1295,207,0.862184,292,190,0.605809,0.993197
4,1401,280,0.833432,0.90,1399,276,0.835224,347,271,0.561489,0.994269


In [10]:
# 테스트 데이터에 대한 error detection결과 및 precision, recall, F1 score 산출
new_TP = sum(train_test_df['Test_TP'].values)
new_FP = sum(train_test_df['Test_FP'].values)

new_precision = new_TP / (new_TP + new_FP)
new_recall = new_TP / len(test_pos_embedding)
new_f1 = (2*new_precision*new_recall) / (new_precision+new_recall)

print('TP : ', new_TP)
print('FP : ', new_FP)
print('precision : ', new_precision)
print('recall : ', new_recall)
print('F1 : ', new_f1)

print(f'\nDetected {len(test_neg_embedding) - new_FP} error triples')

TP :  4991
FP :  4936
precision :  0.5027702226251637
recall :  0.9982
F1 :  0.6687211093990754

Detected 64 error triples
